# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import googlemaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file.dropna()
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Melbourne,1,US,1619905204,57,28.0836,-80.6081,82.40,11.50
1,1,Tuktoyaktuk,90,CA,1619904796,92,69.4541,-133.0374,14.00,5.75
2,2,Olinda,75,BR,1619905230,69,-8.0089,-34.8553,80.60,9.22
3,3,Griffith,1,US,1619905230,36,41.5284,-87.4237,80.60,19.57
4,4,Grand-Santi,73,GF,1619905231,97,4.2500,-54.3833,74.35,2.68
...,...,...,...,...,...,...,...,...,...,...
121,121,Carnarvon,100,AU,1619905106,64,-24.8667,113.6333,68.00,10.36
122,122,Anadyr,40,RU,1619905263,79,64.7500,177.4833,14.00,11.18
123,123,Ilulissat,18,GL,1619905264,42,69.2167,-51.1000,46.40,12.66
124,124,Kwinana,0,AU,1619905264,100,-32.2500,115.7667,48.20,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)

In [15]:
locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [16]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
temp_file=weather_file.loc[(weather_file['Max Temp']>=70)&(weather_file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather



,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
98,98,Saint-Philippe,0,RE,1619905257,69,-21.3585,55.7679,75.2,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
possible_hotels = possible_vacay.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""
possible_hotels

,City,Country,Lat,Lng,Hotel Name
98,Saint-Philippe,RE,-21.3585,55.7679,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    json.dumps(hotel_name, indent=4, sort_keys=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
possible_hotels

,City,Country,Lat,Lng,Hotel Name
98,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

hfigure.add_layer(markers)

# Display Map
hfigure


Figure(layout=FigureLayout(height='420px'))